## NLP Setup

In [ ]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

In [ ]:
from app.classes.units.unit_type import UnitType
from app.classes.units.all_units import unit_type_dict

f = UnitType.BEFORE
print(type(f), isinstance(f, UnitType))

ut = UnitType['FROM']

print(ut)

x = unit_type_dict[ut]

print(x)

#x = unit_type_dict[]


In [3]:
from app.classes.pattern_classes.all_pattern_classes import *

a = [BeforeDate(), BeforeDate()]

d_a = []
for x in a:
    if type(x) not in [type(y) for y in d_a]:
        d_a.append(x)

print(d_a)

In [ ]:
from app.classes.grammar.grammar_node import GrammarNode
from app.src.grammar_builder.tree_printer import TreePrinter
from app.src.grammar_builder.grammar_builder_constructor import GrammarBuilderConstructor
from app.classes.pattern_classes.all_pattern_classes import *

gb = GrammarBuilderConstructor.construct()

test_set = GrammarNode('A', [
    GrammarNode('B', [
        GrammarNode('C'),
    ]),
    GrammarNode('D', [
        GrammarNode('E'),
    ]),
])

new_tree = GrammarNode('B', [
    GrammarNode('F'),
])

pattern_classes = get_all_pattern_classes()

pattern_classes = [
    BeforeDate(),
    BeforeEvent(),
    # AfterDate(),
    # WithinTimespanEvent(),
    # TimespanBeforeEvent(),
]

result = gb.build(pattern_classes)


#gb._rec_merge(test_set, new_tree)

tp = TreePrinter()

tp.print(result)

## Generating Sym

In [ ]:
from app.templates.template_getter import get_template

c_id = 'prop'
is_raw = True

if is_raw:
    folder = f'{c_id}/t_raw'
    c_id = f'{c_id}_raw'
else:
    folder = f'{c_id}/t'

contract = get_template(c_id)
sym_spec = contract.to_sym()

FILEPATH = f'./app/templates/{folder}/{c_id}.txt'

f = open(FILEPATH, 'w')
f.write(sym_spec)
f.close()

## Test Bench

In [ ]:
from app.console_lib.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

In [ ]:
val = 'I arrived within 2 weeks of the project being completed.'

doc = nlp(val)

summarizer.summarize(val)